### Raster Analysis with Python

### (Mosaicking and Spectral Indices)

In [7]:
import os
import rasterio as rio
from rasterio.merge import merge as r_merge

In [8]:
image_dir = r'.\images\result'
files = [item for item in os.listdir(image_dir) if item.endswith('.tif')]

files

#perform mosaic
src_files = [rio.open(os.path.join(image_dir, file)) for file in files]
meta_data = src_files[0].meta.copy()

mosaic, out_transform = r_merge(src_files)

meta_data.update({
    "driver": 'GTIFF',
    "transform": out_transform,
    "height": mosaic.shape[1],
    "width": mosaic.shape[2]
})

output_dir = r'.\output'
os.makedirs(output_dir, exist_ok=True)

merged_file = os.path.join(output_dir, "merged.tif")

with rio.open(merged_file, "w", **meta_data) as dst:
    dst.write(mosaic)



### Spectral Indices

In [9]:
import geopandas as gpd
from rasterio.mask import mask as r_mask

In [10]:
def simplify_names(path):
    items = os.listdir(path)
    return [item for item in items if item.endswith('.TIF')]




def crop_raster_to_shapefile(input_raster, input_shapefile, output_raster):

   with rio.open(input_raster) as src:
        
       # read the shapefile with geopandas
        gdf = gpd.read_file(input_shapefile)

        out_meta = src.meta.copy()


        cropped_image, cropped_image_transform = r_mask(src, gdf.geometry, crop=True)

        
        # print(cropped_image)

        # print(out_meta)

        out_meta.update({
            'driver': 'GTiff',
            'height': cropped_image.shape[1],
            'width': cropped_image.shape[2],
            'transform': cropped_image_transform,
            'nodata': 0,
             
        })

        with rio.open(output_raster,  'w', **out_meta) as dst:
            dst.write(cropped_image)



      


In [11]:
import os
images_folders = r'.\images\rasters'
cropped_folders = r'.\images\cropped'
roi_path = r'.\shp\roi_3.shp'

os.makedirs(cropped_folders, exist_ok=True)

for image in simplify_names(images_folders):
    input_raster = os.path.join(images_folders, image)
    output_raster = os.path.join(cropped_folders, image)


    crop_raster_to_shapefile(input_raster, roi_path, output_raster)
    print(f'>>>>> Processed: {output_raster}')

['Band1.TIF', 'Band2.TIF', 'Band3.TIF', 'Band4.TIF', 'Band5.TIF', 'Band6.TIF', 'Band7.TIF']
>>>>> Processed: .\images\cropped\Band1.TIF
>>>>> Processed: .\images\cropped\Band2.TIF
>>>>> Processed: .\images\cropped\Band3.TIF
>>>>> Processed: .\images\cropped\Band4.TIF
>>>>> Processed: .\images\cropped\Band5.TIF
>>>>> Processed: .\images\cropped\Band6.TIF
>>>>> Processed: .\images\cropped\Band7.TIF


### NDVI

NDVI = ((NIR - Red)/ (NIR + Red))

In [13]:
band1 = rio.open('.\images\cropped\Band1.TIF')
band2 = rio.open('.\images\cropped\Band2.TIF')
band3 = rio.open('.\images\cropped\Band3.TIF')
band4 = rio.open('.\images\cropped\Band4.TIF')
band5 = rio.open('.\images\cropped\Band5.TIF')
band6 = rio.open('.\images\cropped\Band6.TIF')
band7 =rio.open('.\images\cropped\Band7.TIF')

<>:1: SyntaxWarning: invalid escape sequence '\i'
<>:2: SyntaxWarning: invalid escape sequence '\i'
<>:3: SyntaxWarning: invalid escape sequence '\i'
<>:4: SyntaxWarning: invalid escape sequence '\i'
<>:5: SyntaxWarning: invalid escape sequence '\i'
<>:6: SyntaxWarning: invalid escape sequence '\i'
<>:7: SyntaxWarning: invalid escape sequence '\i'
<>:1: SyntaxWarning: invalid escape sequence '\i'
<>:2: SyntaxWarning: invalid escape sequence '\i'
<>:3: SyntaxWarning: invalid escape sequence '\i'
<>:4: SyntaxWarning: invalid escape sequence '\i'
<>:5: SyntaxWarning: invalid escape sequence '\i'
<>:6: SyntaxWarning: invalid escape sequence '\i'
<>:7: SyntaxWarning: invalid escape sequence '\i'
C:\Users\Owner\AppData\Local\Temp\ipykernel_6532\491331600.py:1: SyntaxWarning: invalid escape sequence '\i'
  band1 = rio.open('.\images\cropped\Band1.TIF')
C:\Users\Owner\AppData\Local\Temp\ipykernel_6532\491331600.py:2: SyntaxWarning: invalid escape sequence '\i'
  band2 = rio.open('.\images\crop

In [14]:
# Read all the bands and convert them to float(64bit) type
aerosol_band = band1.read(1).astype("float64")
blue_band = band2.read(1).astype("float64")
green_band = band3.read(1).astype("float64")
red_band = band4.read(1).astype("float64")
nir_band = band5.read(1).astype("float64")
swir_1_band = band6.read(1).astype("float64")
swir_2_band = band7.read(1).astype("float64")


In [15]:
ndvi = ((nir_band - red_band)/ (nir_band + red_band))

# save generated ndvi
meta = band1.meta.copy()

meta.update({
    "count": 1,
    "dtype": "float64"
})

spectral_indices_folder = './indices'
os.makedirs(spectral_indices_folder, exist_ok=True)

ndvi_file = os.path.join(spectral_indices_folder, "ndvi.tif")

with rio.open(ndvi_file, "w", **meta) as dst:
    dst.write(ndvi, 1)


C:\Users\Owner\AppData\Local\Temp\ipykernel_6532\3184137583.py:1: RuntimeWarning: invalid value encountered in divide
  ndvi = ((nir_band - red_band)/ (nir_band + red_band))


### Calculating ndwi

NDWI = (Green - NIR)/(Green + NIR)

In [19]:
import numpy as np
ndwi = np.where(
    (green_band + nir_band) == 0, 0,
    ((green_band - nir_band)/ (green_band + nir_band)))



# save generated ndvi
meta = band1.meta.copy()

meta.update({
    "count": 1,
    "dtype": "float64"
})

spectral_indices_folder = './indices'
os.makedirs(spectral_indices_folder, exist_ok=True)

ndwi_file = os.path.join(spectral_indices_folder, "ndwi.tif")

with rio.open(ndwi_file, "w", **meta) as dst:
    dst.write(ndwi, 1)

C:\Users\Owner\AppData\Local\Temp\ipykernel_6532\1346042861.py:4: RuntimeWarning: invalid value encountered in divide
  ((green_band - nir_band)/ (green_band + nir_band)))


### Calculating NDBI

NDBI = (SWIR - NIR)/(SWIR + NIR)

In [20]:
import numpy as np
ndbi = np.where(
    (swir_1_band + nir_band) == 0, 0,
    ((swir_1_band - nir_band)/ (swir_1_band + nir_band)))



# save generated ndvi
meta = band1.meta.copy()

meta.update({
    "count": 1,
    "dtype": "float64"
})

spectral_indices_folder = './indices'
os.makedirs(spectral_indices_folder, exist_ok=True)

ndbi_file = os.path.join(spectral_indices_folder, "ndbi.tif")

with rio.open(ndbi_file, "w", **meta) as dst:
    dst.write(ndbi, 1)

C:\Users\Owner\AppData\Local\Temp\ipykernel_6532\427273684.py:4: RuntimeWarning: invalid value encountered in divide
  ((swir_1_band - nir_band)/ (swir_1_band + nir_band)))


### CALCULATING BAI

### BAI = 1/((0.1 - RED) ^ 2 + (0.06 - NIR) ^2)